In [1]:

import json
import pennylane as qml

import pennylane.numpy as np

symbols = ["H", "H", "H"]


def h3_ground_energy(bond_length):

    """
    Uses VQE to calculate the ground energy of the H3+ molecule with the given bond length.

    Args:
        - bond_length(float): The bond length of the H3+ molecule modelled as an
        equilateral triangle.
    Returns:
        - Union[float, np.tensor, np.array]: A float-like output containing the ground
        state of the H3+ molecule with the given bond length.
    """
    # generate the molecular Hamiltonian for H3+


    #dataset = qml.data.load('qchem',molname="H3+", bondlength=bond_length)[0]
    #hamiltonian, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)

    bond_length_au = bond_length
    geometry = np.array([[0, 0, 0], [bond_length_au, 0, 0], [bond_length_au/2, (np.sqrt(3)*bond_length_au)/2, 0.0]])
    hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=1)

    hf_state = qml.qchem.hf_state(electrons = 2, orbitals = qubits)

    num_wires = qubits

    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev)
    def exp_energy(state):
      qml.BasisState(np.array(state), wires=range(num_wires))
      return qml.expval(hamiltonian)

    def ansatz(params):
        qml.BasisState(hf_state, wires=range(num_wires))
        qml.DoubleExcitation(params[0], wires=[0,1,2,3])
        qml.DoubleExcitation(params[1], wires=[0,1,4,5])

    @qml.qnode(dev)
    def cost_function(params):
        ansatz(params)
        return qml.expval(hamiltonian)

    max_iterations = 200
    conv_tol = 1e-09
    step_size = 0.4

    opt = qml.GradientDescentOptimizer(step_size)

    #theta = np.random.uniform(low=0, high=2 * np.pi, size=12, requires_grad=True)
    theta = np.array([0.0,0.0], requires_grad = True)
    energy = [cost_function(theta)]
    angle = [theta]


    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_function, theta)

        energy.append(cost_function(theta))
        angle.append(theta)

        conv = np.abs(energy[-1] - prev_energy)

        if n % 2 == 0:
            print(
                "Iteration = {:},  Energy = {:.8f} Ha".format(n, energy[-1])
            )

        if conv <= conv_tol:
            break

    print()
    print("Final convergence parameter = {:.8f} Ha".format(conv))
    print("Number of iterations = ", n)
    print("Final value of the ground-state energy = {:.8f} Ha".format(energy[-1]))
    print()
    print("Final circuit parameters = \n", theta)

    return energy[-1]


# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = h3_ground_energy(ins)
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct ground energy"


# These are the public test cases
test_cases = [
    ('1.5', '-1.232574'),
    ('0.8', '-0.3770325')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.5'...
Iteration = 0,  Energy = -1.22373792 Ha
Iteration = 2,  Energy = -1.23121801 Ha
Iteration = 4,  Energy = -1.23236688 Ha
Iteration = 6,  Energy = -1.23254272 Ha
Iteration = 8,  Energy = -1.23256964 Ha
Iteration = 10,  Energy = -1.23257377 Ha
Iteration = 12,  Energy = -1.23257440 Ha
Iteration = 14,  Energy = -1.23257450 Ha
Iteration = 16,  Energy = -1.23257451 Ha
Iteration = 18,  Energy = -1.23257452 Ha

Final convergence parameter = 0.00000000 Ha
Number of iterations =  18
Final value of the ground-state energy = -1.23257452 Ha

Final circuit parameters = 
 [0.15455614 0.15501789]
Correct!
Running test case 1 with input '0.8'...
Iteration = 0,  Energy = -0.37474085 Ha
Iteration = 2,  Energy = -0.37697796 Ha
Iteration = 4,  Energy = -0.37703137 Ha
Iteration = 6,  Energy = -0.37703265 Ha
Iteration = 8,  Energy = -0.37703268 Ha

Final convergence parameter = 0.00000000 Ha
Number of iterations =  9
Final value of the ground-state energy = -0.37703268 